## Finilizing Data Generation for 0D Reactor

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

pd.options.mode.chained_assignment = None  # default='warn'

from PCAfold import PCA as PCAA

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_POD/'
# TestDir    = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_Test/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

NVarsRed           = 7

tStratch           = 1.
SOLVER             = 'BDF'

ValidPerc          = 25.

FixedMinVal        = 1.e-14

POD_NAME           = '1'
iPC                = 0

if (POD_NAME == 'All'):
    NPODsFinal = 128
    PODType    = 'All'
else:
    NPODsFinal = 64
    PODType    = 'OneByOne'
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType))
except:
    pass

In [3]:
FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
Data         = pd.read_csv(FileName, header=0)
SimIdxs      = Data.to_numpy(int)


FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
Data         = pd.read_csv(FileName)
ICVecs       = Data.to_numpy()
NICs         = len(ICVecs)

In [4]:
if (POD_NAME == 'All'):
    
    Cols   = []
    NewFlg = True
    for iC in range(NICs):

        Nts                  = SimIdxs[iC,1]-SimIdxs[iC,0]
        NtsNow               = Nts



        FileName             = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
        Data                 = pd.read_csv(FileName, header=0)
        DataTemp             = Data[['PC_'+str(iPC+1) for iPC in range(NVarsRed)]]


        for iPC in range(NVarsRed):
            if (np.abs( (DataTemp['PC_'+str(iPC+1)][0] - DataTemp['PC_'+str(iPC+1)][len(Data)-1])/DataTemp['PC_'+str(iPC+1)][0] ) > 1.e-3): 
                if (NewFlg):
                    DataInput        = DataTemp
                    NewFlg           = False
                else:
                    DataInput        = pd.concat([DataInput, DataTemp['PC_'+str(iPC+1)]], axis=1)
                #Cols.append(str(iC+1))
                Cols = Cols+[str(iC+1)+'_PC_'+str(iPC+1)]

    tVec              = Data['t']
    
    yMat              = DataInput.to_numpy()
    
else:
    
    Cols   = []
    NewFlg = True
    for iC in range(NICs):

        Nts                  = SimIdxs[iC,1]-SimIdxs[iC,0]
        NtsNow               = Nts



        FileName             = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
        Data                 = pd.read_csv(FileName, header=0)
        DataTemp             = Data['PC_'+str(iPC+1)]

        if (np.abs( (DataTemp[0] - DataTemp[len(Data)-1])/DataTemp[0] ) > 1.e-3):
            DataICTemp  = Data[['PC_'+str(iPC+1) for iPC in range(NVarsRed)]].iloc[0]
            
            if (NewFlg):
                DataInput        = DataTemp
                DataIC           = DataICTemp
                NewFlg           = False
            else:
                DataInput        = pd.concat([DataInput, DataTemp], axis=1)
                DataIC           = pd.concat([DataIC, DataICTemp], axis=1)
            Cols.append(str(iC+1))

    tVec              = Data['t']
    DataInput.columns = Cols

    yMat              = DataInput.to_numpy()
    
    DataIC            = DataIC.T.reset_index(drop=True, inplace=False)

In [5]:
ICs = DataIC.to_numpy()

In [6]:
DataIC

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7
0,-36.174606,-0.144061,-0.098441,0.005528,-0.030926,-0.011746,0.014059
1,-20.741896,0.025144,0.113525,0.016058,-0.005369,-0.003479,0.004089
2,-4.009643,0.193755,0.371264,0.034843,0.021445,0.004328,-0.006086
3,-13.918271,0.307196,-0.182637,-0.082153,0.018420,0.016317,-0.009158
4,-1.329393,0.176421,0.495972,0.059862,0.023068,0.002125,-0.005825
...,...,...,...,...,...,...,...
495,-20.091448,0.213422,-0.218340,-0.073415,0.006625,0.010978,-0.004058
496,-22.935139,-0.047123,0.174120,0.038497,-0.011907,-0.008410,0.007563
497,-4.497073,0.165678,0.407337,0.045794,0.019268,0.002296,-0.004802
498,-16.761158,0.270660,-0.211588,-0.081429,0.013389,0.014376,-0.007093


In [7]:
# NPODs      = 128
# ScalingVec = ['none', 'auto', 'std', 'pareto', 'vast', 'range', '0to1']
# #'none', '', 'auto', 'std', 'pareto', 'vast', 'range', '0to1', '-1to1', 'level', 'max', 'poisson', 'vast_2', 'vast_3', 'vast_4'

# plt.figure(figsize=(8,8))
# for j, Scaling in enumerate(ScalingVec):

#     pca        = PCAA(yMat, scaling=Scaling, n_components=int(NPODs))
#     C          = pca.X_center
#     D          = pca.X_scale
#     A          = pca.A[:,0:NPODs].T
#     L          = pca.L
#     AT         = A.T

#     plt.plot(L, label=Scaling)
    
# plt.ylim([1.e-14, 1.e0])
# plt.yscale('log')
# plt.legend()

In [8]:
# NPODsVec   = np.arange(30,150,10)
# ScalingVec = ['none', 'auto', 'std', 'pareto', 'vast', 'range', '0to1']
# #'none', '', 'auto', 'std', 'pareto', 'vast', 'range', '0to1', '-1to1', 'level', 'max', 'poisson', 'vast_2', 'vast_3', 'vast_4'

# MaxPerc    = np.zeros((len(NPODsVec), len(ScalingVec)))
# MaxSE      = np.zeros((len(NPODsVec), len(ScalingVec)))
# MeanPerc   = np.zeros((len(NPODsVec), len(ScalingVec)))
# MSE        = np.zeros((len(NPODsVec), len(ScalingVec)))
# for i, NPODs in enumerate(NPODsVec):
#     #print('NPODs = ', NPODs)
    
#     for j, Scaling in enumerate(ScalingVec):

#         pca        = PCAA(yMat, scaling=Scaling, n_components=int(NPODs))
#         C          = pca.X_center
#         D          = pca.X_scale
#         A          = pca.A[:,0:NPODs].T
#         L          = pca.L
#         AT         = A.T

#         yMat_pca   = ((yMat - C)/D).dot(AT)
#         yMat_      = (yMat_pca.dot(A))*D + C
        
#         MaxPerc[i,j]    = np.max(abs((yMat - yMat_)/yMat)*100)
#         MaxSE[i,j]      = np.max((yMat - yMat_)**2)
#         MeanPerc[i,j]   = np.mean(abs((yMat - yMat_)/yMat)*100)
#         MSE[i,j]        = np.mean((yMat - yMat_)**2)

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MaxPerc, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Max % Error')
# plt.legend()
# plt.yscale('log')

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MaxSE, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Max Squared Error')
# plt.legend()
# plt.yscale('log')

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MeanPerc, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Mean % Error')
# plt.legend()
# plt.yscale('log')

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MSE, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Mean Squared Error')
# plt.legend()
# plt.yscale('log')

In [9]:
pca        = PCAA(yMat, scaling='pareto', n_components=NPODsFinal, nocenter=False)
C          = pca.X_center
D          = pca.X_scale
A          = pca.A[:,0:NPODsFinal].T
L          = pca.L
AT         = A.T
print('[PCA] Shape of A        = ', A.shape)
print('[PCA] ')

[PCA] Shape of A        =  (64, 500)
[PCA] 


In [10]:
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/')
except:
    pass

FileName    = OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/A.csv'
np.savetxt(FileName, A, delimiter=',')

FileName    = OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/C.csv'
np.savetxt(FileName, C, delimiter=',')

FileName    = OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/D.csv'
np.savetxt(FileName, D, delimiter=',')

In [11]:
#yMat_pca    = pca.transform(yMat, nocenter=False)
yMat_pca   = ((yMat - C)/D).dot(AT)

yMat_      = (yMat_pca.dot(A))*D + C
print('[PCA] Shape of yMat_pca = ', yMat_pca.shape)
print('[PCA]  Max % Error = ', np.max(abs((yMat - yMat_)/yMat)*100))
print('[PCA]  Max      SE = ', np.max((yMat - yMat_)**2))
print('[PCA] Mean % Error = ', np.mean(abs((yMat - yMat_)/yMat)*100))
print('[PCA]          MSE = ', np.mean((yMat - yMat_)**2))

[PCA] Shape of yMat_pca =  (5000, 64)
[PCA]  Max % Error =  6392.215464483998
[PCA]  Max      SE =  1.6181362374901849
[PCA] Mean % Error =  0.13756997198665222
[PCA]          MSE =  0.0001315435846927278


In [12]:
%matplotlib qt

iVar    = 40

fig = plt.figure()
ax  = fig.add_subplot(projection='3d')
ax.scatter(ICs[:,0], ICs[:,1], A[iVar,:], c='k')
ax.set_xlabel(r'$\eta_{'+str(1)+'}$')
ax.set_ylabel(r'$\eta_{'+str(2)+'}$')
ax.set_zlabel(r'$\Psi_{'+str(iVar+1)+'}$')

Text(0.5, 0, '$\\Psi_{41}$')

In [13]:
A.shape

(64, 500)

In [14]:
plt.figure(figsize=(10,8))
plt.plot(tVec, yMat_pca[:,0], '-k')
plt.xscale('log')
#plt.xlim([1.e-6,1.e-4])

In [15]:
plt.figure(figsize=(10,8))
plt.plot(tVec, yMat[:,0], '-k')
plt.plot(tVec, yMat_[:,0], '-r')
plt.xscale('log')
plt.xlim([1.e-6,1.e-2])

(1e-06, 0.01)

In [16]:
Vars             = ['POD_'+str(iPOD+1) for iPOD in range(NPODsFinal)]
Data             = pd.DataFrame(yMat_pca, columns=Vars)
tVec[tVec == 0.] = FixedMinVal
Data['t']        = tVec
Data['log(t)']   = np.log(tVec)
Data['log10(t)'] = np.log10(tVec)


Data

,POD_1,POD_2,POD_3,POD_4,POD_5,POD_6,POD_7,POD_8,POD_9,POD_10,...,POD_58,POD_59,POD_60,POD_61,POD_62,POD_63,POD_64,t,log(t),log10(t)
0,-85.543013,7.403874,2.603384,1.407501,0.838372,0.547297,0.383116,-0.256796,-0.183911,-0.146538,...,-0.001747,-0.000257,0.000754,0.000777,0.000474,-0.000639,-0.000165,1.000000e-14,-32.236191,-14.000000
1,-85.543013,7.403874,2.603384,1.407501,0.838372,0.547297,0.383116,-0.256796,-0.183911,-0.146538,...,-0.001747,-0.000257,0.000754,0.000777,0.000474,-0.000639,-0.000165,1.000000e-12,-27.631021,-12.000000
2,-85.543013,7.403874,2.603384,1.407501,0.838372,0.547297,0.383116,-0.256796,-0.183911,-0.146538,...,-0.001747,-0.000257,0.000754,0.000777,0.000474,-0.000639,-0.000165,1.097499e-12,-27.537987,-11.959596
3,-85.543013,7.403874,2.603384,1.407501,0.838372,0.547297,0.383116,-0.256796,-0.183911,-0.146538,...,-0.001747,-0.000257,0.000754,0.000777,0.000474,-0.000639,-0.000165,1.204504e-12,-27.444954,-11.919192
4,-85.543013,7.403874,2.603384,1.407501,0.838372,0.547297,0.383116,-0.256796,-0.183911,-0.146538,...,-0.001747,-0.000257,0.000754,0.000777,0.000474,-0.000639,-0.000165,1.321941e-12,-27.351920,-11.878788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,146.324553,17.941582,-5.353119,2.610704,-1.909663,0.833938,-0.550687,-0.386185,0.272872,-0.115700,...,0.003889,0.072228,0.055342,-0.019433,0.020514,-0.007175,-0.000193,9.887808e-03,-4.616453,-2.004900
4996,146.324393,17.941543,-5.353199,2.610701,-1.909707,0.833930,-0.550715,-0.386154,0.272938,-0.115691,...,0.003902,0.072443,0.055522,-0.019494,0.020574,-0.007216,-0.000198,9.915738e-03,-4.613632,-2.003675
4997,146.324234,17.941503,-5.353279,2.610698,-1.909752,0.833921,-0.550743,-0.386122,0.273005,-0.115683,...,0.003915,0.072658,0.055701,-0.019554,0.020634,-0.007257,-0.000203,9.943746e-03,-4.610811,-2.002450
4998,146.324075,17.941464,-5.353358,2.610695,-1.909796,0.833913,-0.550771,-0.386091,0.273072,-0.115674,...,0.003929,0.072872,0.055879,-0.019615,0.020693,-0.007299,-0.000208,9.971833e-03,-4.607991,-2.001225


In [17]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME)
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/')
except:
    pass

In [18]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)


try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/'+data_id+'/Output.csv', index=False)

In [19]:
DataNoZero           = Data
n_points             = len(DataNoZero)

n_test               = n_points


try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/ext/')
except:
    pass

DataInput  = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataInput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/ext/Input.csv', index=False)

DataOutput = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/ext/Output.csv', index=False)

In [20]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME)
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/')
except:
    pass

In [21]:
data_id    = 'pts'

n_points             = len(DataIC)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)


try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/'+data_id+'/')
except:
    pass

DataInput  = DataIC
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/'+data_id+'/Input.csv', index=False)

DataOutput = pd.DataFrame(A.T, columns=['POD_'+str(i+1) for i in range(A.shape[0])])
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/'+data_id+'/Output.csv', index=False)

In [22]:
n_points             = len(DataIC)

n_test               = n_points

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/ext/')
except:
    pass

DataInput  = DataIC
DataInput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/ext/Input.csv', index=False)

DataOutput = pd.DataFrame(A.T, columns=['POD_'+str(i+1) for i in range(A.shape[0])])
DataOutput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/ext/Output.csv', index=False)

In [23]:
NComp = 2

pca        = PCAA(ICs, scaling='none', n_components=NComp)
C          = pca.X_center
D          = pca.X_scale
A          = pca.A[:,0:NComp].T
L          = pca.L
AT         = A.T
print('[PCA] Shape of A        = ', A.shape)
print('[PCA] ')

[PCA] Shape of A        =  (2, 7)
[PCA] 


In [24]:
#yMat_pca    = pca.transform(yMat, nocenter=False)
ICs_pca   = ((ICs - C)/D).dot(AT)

ICs_      = (ICs_pca.dot(A))*D + C
print('[PCA] Shape of yMat_pca = ', ICs_pca.shape)
print('[PCA]  Max % Error = ', np.max(abs((ICs - ICs_)/ICs)*100))
print('[PCA]  Max      SE = ', np.max((ICs - ICs_)**2))
print('[PCA] Mean % Error = ', np.mean(abs((ICs - ICs_)/ICs)*100))
print('[PCA]          MSE = ', np.mean((ICs - ICs_)**2))

[PCA] Shape of yMat_pca =  (500, 2)
[PCA]  Max % Error =  1.3983139236735714e-10
[PCA]  Max      SE =  2.0194839173657902e-28
[PCA] Mean % Error =  6.943566177273015e-13
[PCA]          MSE =  1.1762284378358283e-30


In [25]:
A

array([[-9.99866035e-01, -1.33909630e-02, -9.16440952e-03,
         5.23132848e-04, -1.80214885e-03, -7.24742152e-04,
         7.49483211e-04],
       [ 1.54922455e-03,  4.56583459e-01, -8.58934410e-01,
        -2.26626746e-01,  2.75185334e-02,  3.55608515e-02,
        -1.94755745e-02]])